In [3]:
# Loading the dataset 
import pandas as pd
brick_index_value = pd.read_csv('./bricks_index_values.csv')
brick_rp_distances = pd.read_csv('./brick_rp_distances.csv')
#distances= pd.read_csv('./distances.xlsx')
pfizer =pd.read_csv('./Pfitzer10-100.csv', sep=';')